In [ ]:
from section_retrievers import EdgarRetriever

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader

import chromadb
from chromadb.config import Settings, DEFAULT_DATABASE, DEFAULT_TENANT

In [9]:
loader = CSVLoader(
    file_path='../data/edgar_filings.csv',
    csv_args={'delimiter': ',',
              'quotechar': '"',
              'fieldnames': ['company', 'date', 'section', 'text']},
)

data=loader.load()

In [6]:
chroma_client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

[18:32:42] INFO     Anonymized telemetry enabled. See              ]8;id=719177;file:///home/neal/nealsfiles/stock_rag/stock_advisor_model/venv/lib/python3.10/site-packages/chromadb/telemetry/product/posthog.py\posthog.py]8;;\:]8;id=245447;file:///home/neal/nealsfiles/stock_rag/stock_advisor_model/venv/lib/python3.10/site-packages/chromadb/telemetry/product/posthog.py#20\20]8;;\
                    https://docs.trychroma.com/telemetry for more               
                    information.                                                
           INFO     Anonymized telemetry enabled. See              ]8;id=161643;file:///home/neal/nealsfiles/stock_rag/stock_advisor_model/venv/lib/python3.10/site-packages/chromadb/telemetry/product/posthog.py\posthog.py]8;;\:]8;id=118171;file:///home/neal/nealsfiles/stock_rag/stock_advisor_model/venv/lib/python3.10/site-packages/chromadb/telemetry/product/posthog.py#20\20]8;;\
                    https://docs.trychroma.com/telemetry f